In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
CREATE OR REPLACE SCHEMA UTILS;
USE SCHEMA UTILS;

CREATE OR REPLACE STAGE ML_STAGE
  DIRECTORY = (ENABLE = TRUE)
  ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
from snowflake.snowpark import types as T
from snowflake.snowpark import functions as F
from snowflake import snowpark

def uc01_feature_engineering(session: snowpark.Session, db: str, sc:str, cur_date: datetime, table_name: str):

    # Function to create features that define a profile of customer behavior based on latest purchases
    # Features created for a given date (cur_date)
    # New features added into table (table_name)

    table_name = f'{db}.{sc}.{table_name}'
    
    # Load data
    customers_tbl = '.'.join([db, sc,'CUSTOMERS'])
    sales_tbl = '.'.join([db, sc,'SALES'])
    feedback_tbl = '.'.join([db, sc,'FEEDBACK_SENTIMENT'])
    
    customers_df = session.table(customers_tbl)
    sales_df = session.table(sales_tbl)
    sales_df_last_tran = session.table(sales_tbl)

    feedback_sentiment_df = session.table(feedback_tbl)    

    # we are only doing feature engineering for transactions before cur_date
    
    sales_df_last_tran = sales_df_last_tran.filter(F.col("transaction_date") < F.lit(cur_date))

    sales_df = sales_df.filter(F.col("transaction_date") < F.lit(cur_date ))
        
    # count only feedback before cur_date
    
    feedback_sentiment_df = feedback_sentiment_df.filter(F.col("chat_date") < F.lit(cur_date))
    
    sales_agg_df = (
        sales_df_last_tran.group_by("customer_id")
        .agg(
            F.max("transaction_date").alias("last_purchase_date"),
            F.sum("total_amount").alias("total_customer_value")
        )
    )
    
    def custom_column_naming(input_col, agg, window):
        return f"{agg}_{input_col}_{window.replace('-', 'past_')}"
                                                   
    sales_agg_orders_df = sales_df.analytics.time_series_agg(
            time_col="transaction_date",
            aggs={"total_amount": ["SUM", "COUNT"]},
            windows=["-7D","-1MM", "-2MM", "-3MM"],
            sliding_interval="1D",
            group_by=["CUSTOMER_ID"],
            col_formatter = custom_column_naming)

    sales_agg_last_purchase = sales_agg_df.join(
        sales_agg_orders_df,
        (sales_agg_df.last_purchase_date == sales_agg_orders_df.transaction_date) &
        (sales_agg_df.CUSTOMER_ID == sales_agg_orders_df.CUSTOMER_ID),
        "left").select(
            sales_agg_df["customer_id"].alias("CUSTOMER_ID"),
            sales_agg_df["total_customer_value"],
            sales_agg_df["last_purchase_date"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_7D"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_1MM"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_2MM"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_3MM"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_7D"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_1MM"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_2MM"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_3MM"]
        )

    #  feedback data

    latest_feedback_df = (feedback_sentiment_df.group_by("customer_id")
            .agg(F.max("chat_date").alias("chat_date")))
    
    feedback_agg_df = feedback_sentiment_df.analytics.moving_agg(
            aggs={"SENTIMENT": ["MIN", "AVG"]},
            window_sizes=[2, 3, 4],
            order_by=["chat_date"],
            group_by=["CUSTOMER_ID"])

    
    feedback_agg_latest_df = latest_feedback_df.join(
        feedback_agg_df, "customer_id", "left").select(
            latest_feedback_df["CUSTOMER_ID"].alias("CUSTOMER_ID"),
            feedback_agg_df["SENTIMENT_MIN_2"],
            feedback_agg_df["SENTIMENT_MIN_3"],
            feedback_agg_df["SENTIMENT_MIN_4"],
            feedback_agg_df["SENTIMENT_AVG_2"],
            feedback_agg_df["SENTIMENT_AVG_3"],
            feedback_agg_df["SENTIMENT_AVG_4"],         
        )
    
    # Join tables
    features_df = (
        customers_df.join(sales_agg_last_purchase, "customer_id", "left")
        .join(feedback_agg_latest_df, "customer_id", "left")
        .select(
            customers_df["customer_id"],
            customers_df["age"],
            customers_df["gender"],
            customers_df["location"],
            customers_df["customer_segment"],
            sales_agg_last_purchase["last_purchase_date"],
            feedback_agg_latest_df["SENTIMENT_MIN_2"],
            feedback_agg_latest_df["SENTIMENT_MIN_3"],
            feedback_agg_latest_df["SENTIMENT_MIN_4"],
            feedback_agg_latest_df["SENTIMENT_AVG_2"],
            feedback_agg_latest_df["SENTIMENT_AVG_3"],
            feedback_agg_latest_df["SENTIMENT_AVG_4"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_7D"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_1MM"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_2MM"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_3MM"],
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_7D"].alias("COUNT_ORDERS_PAST_7D"),
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_1MM"].alias("COUNT_ORDERS_PAST_1MM"),
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_2MM"].alias("COUNT_ORDERS_PAST_2MM"),
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_3MM"].alias("COUNT_ORDERS_PAST_3MM"),
            F.datediff("day", sales_agg_df["last_purchase_date"], F.lit(cur_date)).alias("DAYS_SINCE_LAST_PURCHASE"),
            F.lit(cur_date).alias("TIMESTAMP")
        ).filter(sales_agg_df["last_purchase_date"].isNotNull()  # Avoid customers never purchased
        ).dropDuplicates(["customer_id", "TIMESTAMP"])  # Ensure one combination of customer_id and TIMESTAMP

    )
    
    # Fill with 0 those where we have no data (so neutral feedback and zero iterations and amount)
    columns_to_fill = [
        "SENTIMENT_MIN_2", "SENTIMENT_MIN_3", "SENTIMENT_MIN_4", "SENTIMENT_AVG_2",
        "SENTIMENT_AVG_3", "SENTIMENT_AVG_4",
        "SUM_TOTAL_AMOUNT_PAST_7D", "SUM_TOTAL_AMOUNT_PAST_1MM", "SUM_TOTAL_AMOUNT_PAST_2MM", "SUM_TOTAL_AMOUNT_PAST_3MM",
        "COUNT_ORDERS_PAST_7D", "COUNT_ORDERS_PAST_1MM", "COUNT_ORDERS_PAST_2MM", "COUNT_ORDERS_PAST_3MM"
    ]
    
    for column in columns_to_fill:
        features_df = features_df.fillna({column: 0})
    
    # Write to Snowflake Table
    features_df.write.mode("append").save_as_table(table_name)

    print (f'Created table {table_name}')

session.sproc.register(
    func=uc01_feature_engineering,
    name="uc01_feature_engineering_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=T.StringType()
)


In [ ]:
def uc01_label_churn(session: snowpark.Session, db: str, sc:str, baseline_table: str, output_table: str,  
                     num_days_churn: int):

    # Load baseline features dataset
    baseline_table = f'{db}.{sc}.{baseline_table}'
    output_table = f'{db}.{sc}.{output_table}'
    
    baseline_df = session.table(baseline_table)

    # Load sales dataset
    sales_df = session.table(f'{db}.{sc}.SALES')

    # Filter sales to retain only customer ID and transaction date
    sales_filtered = sales_df.select(F.col("CUSTOMER_ID"), F.col("TRANSACTION_DATE"))

    # Find the next transaction date for each (CUSTOMER_ID, TIMESTAMP)
    next_transaction_df = (
        baseline_df
        .join(sales_filtered, "CUSTOMER_ID", "left")
        .filter(F.col("TRANSACTION_DATE") >F.col("LAST_PURCHASE_DATE"))
        .group_by(F.col("CUSTOMER_ID"), F.col("TIMESTAMP"))
        .agg(F.min("TRANSACTION_DATE").alias("NEXT_TRANSACTION_DATE"))
    )

    # Join back with the baseline dataset to compute CHURNED
    final_df = (
        baseline_df
        .join(next_transaction_df, ["CUSTOMER_ID", "TIMESTAMP"], "left")
        .select(
            baseline_df["*"],
            F.when(
                (F.col("NEXT_TRANSACTION_DATE").is_null()) |
                ((F.col("NEXT_TRANSACTION_DATE") - F.col("LAST_PURCHASE_DATE")) > num_days_churn),
                1
            ).otherwise(0).alias("CHURNED"),
            F.col("NEXT_TRANSACTION_DATE")
        )
    )

    # Save the final labeled dataset
    final_df.write.mode("overwrite").save_as_table(output_table)

session.sproc.register(
    func=uc01_label_churn,
    name="uc_01_label_churn_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=T.StringType()
)

In [ ]:
def uc01_update_label_churn(session: snowpark.Session, db: str, sc:str, baseline_table: str,  
                     num_days_churn: int):

    # Load baseline features dataset
    baseline_table = f'{db}.{sc}.{baseline_table}'
    
    baseline_df = session.table(baseline_table)

    # Load sales dataset
    sales_df = session.table(f'{db}.{sc}.SALES')

    # Filter sales to retain only customer ID and transaction date
    sales_filtered = sales_df.select(F.col("CUSTOMER_ID"), F.col("TRANSACTION_DATE"))

    # Find the next transaction date for each (CUSTOMER_ID, TIMESTAMP)
    next_transaction_df = (
        baseline_df
        .join(sales_filtered, "CUSTOMER_ID", "left")
        .filter(F.col("TRANSACTION_DATE") >F.col("LAST_PURCHASE_DATE"))
        .group_by(F.col("CUSTOMER_ID"), F.col("TIMESTAMP"))
        .agg(F.min("TRANSACTION_DATE").alias("NEXT_TX_DATE"))
    )

    # Join back with the baseline dataset to compute CHURNED
    final_df = (
        baseline_df
        .join(next_transaction_df, ["CUSTOMER_ID", "TIMESTAMP"], "left")
        .select(
            baseline_df["CUSTOMER_ID"],
            baseline_df["TIMESTAMP"],
            next_transaction_df["NEXT_TX_DATE"],
            F.when(
                next_transaction_df["NEXT_TX_DATE"].is_null() |
                ((next_transaction_df["NEXT_TX_DATE"] - baseline_df["LAST_PURCHASE_DATE"]) > num_days_churn),
                1
            ).otherwise(0).alias("CHURNED")
        )    
        .with_column_renamed("NEXT_TX_DATE", "NEXT_TRANSACTION_DATE")

    )

    final_df.write.mode("overwrite").save_as_table('temp_updates')

    update_statement = f"""
        update {baseline_table} c
        set CHURNED = t.CHURNED,
            NEXT_TRANSACTION_DATE = t.NEXT_TRANSACTION_DATE
        from temp_updates t
        where c.CUSTOMER_ID = t.CUSTOMER_ID AND
            c.TIMESTAMP = t.TIMESTAMP
          
        """

    session.sql(update_statement).collect()

session.sproc.register(
    func=uc01_update_label_churn,
    name="uc01_update_label_churn_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=T.StringType()
)